<a href="https://colab.research.google.com/github/ucfilho/marquesgabi_2020/blob/master/ANN_classifica_fotos_fev_04_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import math
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 
from mpl_toolkits.mplot3d import Axes3D, axes3d
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [0]:
#@title Digite os parametros { run: "auto", vertical-output: true }
#@markdown Enter neurons hidden layer
N_Hidden=18  #@param {type:"number"}
#@markdown Activation function

Ativa='logistic'#@param ["tanh", "identity", "logistic","relu"] {allow-input: true}

In [0]:
# funcao normatiza dados
def Normatiza(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max()
        min_x=x[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=(x[i]-a)/b
    return x

In [0]:
# funcao retorna os dados a forma original
def Original(x,x_old):
    strings=list(x)
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

In [5]:
'''
!wget http://ftp.feq.ufu.br/ubirajara/Fotos_Treina_ANN_dez_2019.csv
Transfere='Fotos_Treina_ANN_dez_2019.csv'
'''

"\n!wget http://ftp.feq.ufu.br/ubirajara/Fotos_Treina_ANN_dez_2019.csv\nTransfere='Fotos_Treina_ANN_dez_2019.csv'\n"

In [6]:
# clicar no botao e selecionar o arquivo
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()
Transfere='Fotos_Treina_ANN_fev_2020.csv'
df= pd.read_csv(Transfere) 



Saving Fotos_Treina_ANN_fev_2020.csv to Fotos_Treina_ANN_fev_2020 (1).csv


In [7]:
'''
!wget http://ftp.feq.ufu.br/ubirajara/New_Feature_Cor_ANN_jan_2020.csv
Transfere_Cor='New_Feature_Cor_ANN_jan_2020.csv'
'''

"\n!wget http://ftp.feq.ufu.br/ubirajara/New_Feature_Cor_ANN_jan_2020.csv\nTransfere_Cor='New_Feature_Cor_ANN_jan_2020.csv'\n"

In [8]:
# clicar no botao e selecionar o arquivo
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()
Transfere_Cor='Transfere_Color_ANN_fev_2020.csv'
df_cor= pd.read_csv(Transfere_Cor)

Saving Transfere_Color_ANN_fev_2020.csv to Transfere_Color_ANN_fev_2020.csv


In [0]:
df['Color']=df_cor['Color']
# print(df)

In [10]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
print(df.columns)

Index(['Unnamed: 0', 'Type', 'no.peaks', 'Media_proem', 'area under',
       'Width_peaks', 'Width_peaks_max', 'Width_peaks_min', 'Median', 'Mode',
       'Mean', 'Sd', 'Color'],
      dtype='object')


In [0]:
df['Type'] = df['Type'].map({'buraco':1, 'grao':2, 'indef':3}) 

In [12]:
Colunms=['no.peaks', 'Media_proem', 'area under','Width_peaks',
         'Median', 'Mode',
         'Mean', 'Sd','Color']
#X=df.iloc[:,2:12]
#y=df.iloc[:,1].astype(int)
X=df[Colunms]
#y=df['Type'].astype(int)
y=df['Type']
print(y)

0      1
1      1
2      1
3      1
4      1
      ..
535    3
536    3
537    3
538    3
539    3
Name: Type, Length: 540, dtype: int64


In [13]:
print(df.head())

   Unnamed: 0  Type  no.peaks  Media_proem   area under  Width_peaks  Width_peaks_max  Width_peaks_min      Median    Mode        Mean          Sd   Color
0  0           1     2.0       79.741898    2034.578356  45.937917    236.770833       1.653084         58.121528   0.5325  114.076265  139.521107  0.4800
1  1           1     2.0       205.517361   5425.879514  90.480330    615.336806       0.736145         83.760417   0.3900  303.823364  398.033296  0.4725
2  2           1     3.0       113.449074   3565.716782  62.291306    296.361111       0.518618         123.052083  0.5175  196.389090  184.063587  0.3900
3  3           1     3.0       99.371142    3570.312847  52.065348    295.166667       0.720978         133.888889  0.4800  197.921343  167.187493  0.4350
4  4           1     2.0       58.390046    1297.763310  30.487006    125.434028       3.014673         44.909722   0.5625  73.230710   74.532635   0.3625


In [14]:

# separa os dados variaveis de entrada em normatizados e originais
X_OLD=[]
X_OLD=X.copy()
X=Normatiza(X)
#print(X)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, 
                                                    random_state=25)

In [0]:
# define a rede neural 
#N_Hidden=(18,18,18)


clf = MLPClassifier(solver='lbfgs',activation=Ativa,alpha=1e-5,
                    hidden_layer_sizes=N_Hidden,random_state=0)


In [17]:
# treina a rede neural
clf.fit(X_train, y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=18, learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=0, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [0]:
# usa a rede neural treina em calculos para treino e teste
y_calc_train=clf.predict(X_train)
y_pred=clf.predict(X_test)


In [19]:
# Evaluating the Algorithm
print(confusion_matrix(y_pred, y_test))
print(classification_report(y_pred, y_test))

[[16 10  3]
 [11 18 14]
 [10  9 17]]
              precision    recall  f1-score   support

           1       0.43      0.55      0.48        29
           2       0.49      0.42      0.45        43
           3       0.50      0.47      0.49        36

    accuracy                           0.47       108
   macro avg       0.47      0.48      0.47       108
weighted avg       0.48      0.47      0.47       108



In [20]:
'''
Resultado de final de janeiro
[[3 0 0]
 [4 6 5]
 [1 2 3]]
              precision    recall  f1-score   support

           1       0.38      1.00      0.55         3
           2       0.75      0.40      0.52        15
           3       0.38      0.50      0.43         6

    accuracy                           0.50        24
   macro avg       0.50      0.63      0.50        24
weighted avg       0.61      0.50      0.50        24
'''

'\nResultado de final de janeiro\n[[3 0 0]\n [4 6 5]\n [1 2 3]]\n              precision    recall  f1-score   support\n\n           1       0.38      1.00      0.55         3\n           2       0.75      0.40      0.52        15\n           3       0.38      0.50      0.43         6\n\n    accuracy                           0.50        24\n   macro avg       0.50      0.63      0.50        24\nweighted avg       0.61      0.50      0.50        24\n'

In [21]:
'''
[[5 4 2]
 [2 1 1]
 [1 3 5]]
              precision    recall  f1-score   support

           1       0.62      0.45      0.53        11
           2       0.12      0.25      0.17         4
           3       0.62      0.56      0.59         9

    accuracy                           0.46        24
   macro avg       0.46      0.42      0.43        24
weighted avg       0.54      0.46      0.49        24
'''

'\n[[5 4 2]\n [2 1 1]\n [1 3 5]]\n              precision    recall  f1-score   support\n\n           1       0.62      0.45      0.53        11\n           2       0.12      0.25      0.17         4\n           3       0.62      0.56      0.59         9\n\n    accuracy                           0.46        24\n   macro avg       0.46      0.42      0.43        24\nweighted avg       0.54      0.46      0.49        24\n'

In [22]:
print(y_pred)

[3 1 2 2 3 2 1 3 3 3 2 3 1 2 3 2 3 2 3 2 1 3 1 2 1 2 1 2 3 2 3 3 2 3 1 3 2
 1 3 3 1 3 1 2 3 2 2 2 1 2 3 3 1 2 1 2 3 1 2 3 1 2 2 3 3 2 1 1 2 2 2 3 3 2
 3 2 2 1 3 2 3 2 1 1 3 2 3 3 1 3 2 1 3 2 2 1 1 2 1 3 1 2 2 1 2 2 2 1]
